In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "SNHU2024"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.readAll())

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

# display logo and unique identifier side-by-side at the top
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        html.Center(html.H5('Created by Joseph Veneski -- 10/15/2023')),
        html.Hr(),
    ], style={'width': '50%', 'text-align': 'center', 'display': 'inline-block'}),
    
    html.Div([
        html.Center(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'width': '50%', 'height': '50%'}
            )
        ),
        html.Hr(),
    ], style={'width': '50%', 'text-align': 'center', 'display': 'inline-block'}),
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div([
        dcc.Dropdown(
            id= 'dropdown-menu',
            options=[
                {'label' : 'Water Rescue', 'value' : 'water'},
                {'label' : 'Mountain or Wilderness Rescue', 'value' : 'mountain'},
                {'label' : 'Disaster or Individual Tracking Rescue', 'value' : 'disaster'},
                {'label' : 'Reset <no filters>', 'value' : 'reset'}
            ],
            value='reset'
        ),
        html.Div(id='dd-output-container')
    ]),
        
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native", #for filtering results
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable='single', #make sure we can select rows to pop map
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=5, #can adjust number of results
    ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input(component_id='dropdown-menu', component_property='value')])
def update_dashboard(value):
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    # Rescue filter queries based on client needs
    if value == 'reset':
        dff = pd.DataFrame.from_records(db.readAll())
    elif value == 'water':
        dff = pd.DataFrame.from_records(db.read({
            "breed": {
                "$in": [
                    "Chesapeake Bay Retriever",
                    "Labrador Retriever Mix",
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }))
    elif value == 'mountain':
        dff = pd.DataFrame.from_records(db.read({
            'animal_type' : 'Dog',
            'breed' : {
                '$in': [
                    "German Shepard",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            'sex_upon_outcome' : 'Intact Male',
            'age_upon_outcome_in_weeks': {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }))
    elif value == 'disaster':
        dff = pd.DataFrame.from_records(db.read({
            'animal_type' : 'Dog',
            'breed' : {
                '$in': [
                    "Doberman Pinscher",
                    "German Shepard",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            'sex_upon_outcome' : 'Intact Male',
            'age_upon_outcome_in_weeks': {
                "$gte": 20.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 300.0
                    }
                }
            ]
        }))
    else:
        dff = pd.DataFrame.from_records(db.readAll())
    
    # get rid of the _id column
    dff.drop(columns=['_id'], inplace=True)
    
    # use dictionary
    data = dff.to_dict('records')   
    
    return (data, columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    #add code for chart of your choice (e.g. pie chart) #
    if viewData is None:
        return []
    
    df = pd.DataFrame.from_records(viewData)
    if 'breed' not in df.columns:
        return []
    
    names = df['breed'].value_counts().keys().tolist()
    values = df['breed'].value_counts().tolist()
    return [
        dcc.Graph(            
            figure = px.pie(
                df,
                values = values,
                names = names,
                title='Preferred Animals'
            )
        )    
    ]
    


#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart
    df = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can
    # be converted to a row index here
    if not index:
        return []
    else:
        row = index[0]
        
    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
          center=[30.75,-97.48], zoom=10, children=[
              dl.TileLayer(id="base-layer-id"),
              # Marker with tool tip and popup
              # Column 13 and 14 define the grid coordinates for
              # the map
              # Column 4 defines the breed for the animal
              # Column 9 defines the name of the animal
              dl.Marker(position=[df.iloc[row]['location_lat'],df.iloc[row]['location_long']],
                       children=[
                           dl.Tooltip(df.iloc[row]['breed']),
                           dl.Popup([
                               html.H1("Animal Name"),
                               html.P(df.iloc[row]['name'])
                           ])
                       ])
              ]
              
    )
]

app.run_server(debug=True)


Connection Successful
Dash app running on http://127.0.0.1:12356/
